In [ ]:
# unfreeze a few of the top layers of a frozen model base used for feature extraction and jointly train the new added model and these top layers
# the more specialized feature need in fact to be retrained

In [ ]:
from keras.applications import VGG16 # TODO: try also Xception, Inception V3, ResNet50, VGG16, VGG19, MobileNet and others domain-specific nets
from keras.applications import Xception
from keras.applications import ResNet50
from keras.applications import EfficientNetB2
import os
import csv
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
images_path = os.path.join("..","workspace","images")
train_dir = os.path.join(images_path, "train")
validation_dir = os.path.join(images_path, "eval")
test_dir = os.path.join(images_path, "test")
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [ ]:
# hyperparameters to be set
input_height = 256
input_width = 256
loss_function='mse'
metrics=['mae']
optimizer='rmsprop'
optimizer_learning_rate=1e-4
epochs=5
num_classes=8
batch_size=20
if optimizer == 'rmsprop':
    optimizer=optimizers.RMSprop(learning_rate=optimizer_learning_rate)

In [ ]:
#import pre-trained models
vgg16 = VGG16(weights='imagenet', # weight checkpoint from which we initialize the model
                  include_top=False, # to decide if need to include the densely connected layer
                  input_shape=(input_width, input_height, 3)) #TODO: check order

xception = Xception(weights='imagenet',
                  include_top=False,
                  input_shape=(input_width, input_height, 3))

resnet = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(input_width, input_height, 3))

efficientnetb2 = EfficientNetB2(weights='imagenet',
                  include_top=False,
                  input_shape=(input_width, input_height, 3))

In [ ]:
pretrained_models = [vgg16,xception,resnet,efficientnetb2]
names = ["Vgg16","Xception","Resnet","EfficientNetB2"]

In [ ]:
#generate train,test and validation set
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest') # set right parameters

df_train = pd.read_csv(os.path.join(train_dir, "df_train.csv"))
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

In [ ]:
def build_model(input_conv_base):
    #build the cnn using the pre-trained cnn
    built_model = models.Sequential()
    built_model.add(input_conv_base)
    built_model.add(layers.Flatten())
    built_model.add(layers.Dense(256, activation='relu'))
    built_model.add(layers.Dense(num_classes, activation='softmax'))

    return built_model

In [ ]:
def evaluate_model(model_to_evaluate,test_images):

    model_loss, model_accuracy = model_to_evaluate.evaluate_generator(test_images, steps=2)

    return [model_loss, model_accuracy]

In [ ]:
def plot_accuracy_and_loss(history_to_plot):
    acc = history_to_plot.history['acc']
    val_acc = history_to_plot.history['val_acc']
    loss = history_to_plot.history['loss']
    val_loss = history_to_plot.history['val_loss']
    history_epochs = range(1, len(acc) + 1)
    plt.plot(history_epochs, acc, 'r', label='Training acc')
    plt.plot(history_epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation MAE')
    plt.legend()
    plt.figure()
    plt.plot(history_epochs, loss, 'r', label='Training loss')
    plt.plot(history_epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()


In [ ]:
def write_to_csv(metrics_to_write):
    header = ["conv_base_name","acc","val_acc"]
    with open('TRANSFER_LEARNING_RESULTS.csv', 'a') as file:
        writer = csv.writer(file)
        # write header first
        writer.writerow(header)

        #write all the other metrics
        for metric in metrics_to_write:
            writer.writerow(metric)



In [ ]:
#stores metrics for each fine tuned model
model_metrics = []

#fine tune and evaluate each model
for conv_base_name, conv_base in zip(names,pretrained_models):
    conv_base.trainable = True
    set_trainable = False
    #fine tune
    for layer in conv_base.layers:
        if layer.name == 'block5_conv1': # TODO: decide the layer where to stop
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

        #build model
        model = build_model(conv_base)

        model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=metrics)

        history = model.fit_generator(
              train_generator,
              steps_per_epoch=10,
              epochs=3,
              validation_data=validation_generator,
              validation_steps=5)

        plot_accuracy_and_loss(history)
        cur_metrics = evaluate_model(model,test_generator)
        #build list in this format [conv_base_name , model_loss, model_accuracy
        model_metrics.append([conv_base_name].extend(cur_metrics))

#write metrics to file
write_to_csv(model_metrics)

In [ ]:
# use optimizer with very low learning rate to limit the magnitude of modifications we make

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) # it should not be augmented

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

In [ ]:

# if performances are much wors than validation ones, during hyperparameter optimization (when done) the process has overfitted the validdation set, if so go to a more clear protocol such as Kfold CV
test_loss, test_acc = model.evaluate_generator(test_generator, steps=2)
print('test acc:', test_acc)
print('test loss:', test_loss)